<a href="https://colab.research.google.com/github/trefftzc/partition_COLAB_notebooks/blob/main/partition_openmp_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving the partition problem with a program in C with OpenMP

In [1]:
%%writefile openmpPartition.c
/*
 * sequentialPartition.c
 * Solve the Partition problem sequentially.
 * https://en.wikipedia.org/wiki/Partition_problem
 * This code works for multisets of up to 32 elements
 * The input is expected to be as follows:
 * The first line will contain n, the number of elements in the multiset
 * The remaining n lines will contain the n values, one per line
 */
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

// This functions evaluate a partition of the multiset.
// The values in the multiset are passed as a parameter in array
// n is the size of the multiset
// value is an integer value. The binary code of value
// encodes a partition:
// One partition corresponds to the bits that are 0 in the binary encoding of value
// The other partition corresponds to the bits that are 1 in the binary encoding of value
int evaluatePartition( unsigned int value, int n, int *array) {
  int sum0s = 0;
  int sum1s = 0;
  unsigned int mask = 1;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      sum1s = sum1s + array[i];
    }
    else {
      sum0s = sum0s + array[i];
    }
    mask = mask * 2;
  }
  if (sum0s == sum1s)
     return 1;
  else
     return 0;
}

void printResults(unsigned int value,int n,int *array)
{
  printf("Solution:\n");
  printf("First partition: ") ;
  unsigned int mask = 1;
  int sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      printf("%d ",array[i]);
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  printf(" sum: %d \n",sum);
  printf("Second partition: ") ;
  mask = 1;
  sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) == 0) {
      printf("%d ",array[i]);
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  printf(" sum: %d \n",sum);
}

// The main function
int main() {

  int n;
  int *array;
// The format of the input is
// an integer value n with the size of the multiset
// n integer values with the multiset
  scanf("%d",&n);

  printf("The value of n is %d\n",n);
  array = (int *) malloc (n * sizeof(int));
  for(int i = 0;i < n;i++) {
    scanf("%d",&array[i]);
  }
  printf("The read values are: \n");
  for(int i = 0;i < n;i++) {
    printf("%d ",array[i]);
  }
  printf("\n");
// Calculate the set of the power set
  unsigned int nPartitions = 1;
  for(int i = 0;i < n;i++) {
    nPartitions = nPartitions * 2;
  }
  // printf("The number of possible partitions is: %d\n",nPartitions);
  // Only half of all possible partitions need be examined
  // The second half is symmetrical to the first half
  nPartitions = nPartitions / 2;
 int solutionFound = 0;
  int solution = -1;
  #pragma omp parallel for shared(solutionFound,solution)
  for(unsigned int i = 1;i < (nPartitions);i++) {
    int result = evaluatePartition(i,n,array);
  #pragma omp critical
    if (result == 1) {
      solutionFound = 1;
      solution = i;
    }
  }
  if (solutionFound) {
    printResults(solution, n, array);
  }
  else {
    printf("No solution was found.");
  }
  return 0;
}


Writing openmpPartition.c


This is a test file that has a solution

In [3]:

%%writefile testFile24.Text
24
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
23


Writing testFile24.Text


Compiling the code, and running the code with test file

In [4]:
!gcc openmpPartition.c -o openmpPartition -O3 -fopenmp
!time ./openmpPartition < testFile24.Text

openmpPartition.c: In function ‘main’:
openmpPartition.c:75:3: warning: ignoring return value of ‘scanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   75 |   scanf("%d",&n);
      |   ^~~~~~~~~~~~~~
openmpPartition.c:80:5: warning: ignoring return value of ‘scanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   80 |     scanf("%d",&array[i]);
      |     ^~~~~~~~~~~~~~~~~~~~~
The value of n is 24
The read values are: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 23 
Solution:
First partition: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1  sum: 23 
Second partition: 23  sum: 23 

real	0m0.327s
user	0m0.604s
sys	0m0.008s


Let's create other larger test files and see how long the program takes to execute with those larger files.

In [5]:
%%writefile testFile25.Text
25
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
24

Writing testFile25.Text


In [6]:
!time ./openmpPartition < testFile25.Text

The value of n is 25
The read values are: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 24 
Solution:
First partition: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1  sum: 24 
Second partition: 24  sum: 24 

real	0m0.649s
user	0m1.229s
sys	0m0.004s


In [7]:
%%writefile testFile26.Text
26
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
25

Writing testFile26.Text


In [8]:
!time ./openmpPartition < testFile26.Text

The value of n is 26
The read values are: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 25 
Solution:
First partition: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1  sum: 25 
Second partition: 25  sum: 25 

real	0m1.836s
user	0m2.501s
sys	0m0.009s


In [10]:
%%writefile testFile30.Text
30
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
29

Writing testFile30.Text


In [11]:
!time ./openmpPartition < testFile30.Text

The value of n is 30
The read values are: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 29 
Solution:
First partition: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1  sum: 29 
Second partition: 29  sum: 29 

real	0m22.514s
user	0m41.963s
sys	0m0.034s


The CPU on COLAB has only 2 cores. Hence the speedups will be at most 2.
Try this code on a computer with more cores, and you should see higher speedups.

In [12]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         48 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               AuthenticAMD
  Model name:            AMD EPYC 7B12
    CPU family:          23
    Model:               49
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4499.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm cons
                         tant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pcl
                         mulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c 
                         rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3